In [1]:
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor import EndpointInput
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.model_monitor import CronExpressionGenerator

import sagemaker


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [48]:
%store -r baseline_dataset_uri
%store -r path
%store -r endpoint_name
session = sagemaker.Session()
bucket = session.default_bucket()
baseline_results_uri = f"s3://{bucket}/{path}/baseline-results"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
baseline_dataset_uri

's3://sagemaker-us-east-1-381492271173/monitor/heart/baseline/dataset/validation_with_predictions.csv'

In [24]:
model_quality_monitor = ModelQualityMonitor(
    role='LabRole',
    instance_count=1,
    instance_type="ml.m5.xlarge",
    sagemaker_session=session)

In [5]:
job = model_quality_monitor.suggest_baseline(
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    problem_type="BinaryClassification",
    inference_attribute="prediction",
    probability_attribute="probability",
    ground_truth_attribute="label",
)
job.wait()

INFO:sagemaker:Creating processing-job with name baseline-suggestion-job-2024-03-18-19-19-39-195


............................2024-03-18 19:24:25.382905: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-18 19:24:25.382939: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-03-18 19:24:26.999741: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-03-18 19:24:26.999769: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-03-18 19:24:26.999787: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-0-138-184.ec2.internal): /proc/driver/nvidia/version does not exist
2024-03-18 19:24:27.

In [49]:
endpointInput = EndpointInput(
    endpoint_name=endpoint_name,
    probability_attribute="0",
    probability_threshold_attribute=0.5,
    destination="/opt/ml/processing/input_data",
)


In [50]:
endpoint_name

'sagemaker-scikit-learn-2024-03-21-00-03-05-484'

In [51]:
response = model_quality_monitor.create_monitoring_schedule(
    endpoint_input=endpointInput,
    output_s3_uri=baseline_results_uri,
    problem_type="BinaryClassification",
    ground_truth_input="s3://sagemaker-us-east-1-381492271173/monitor/heart/ground-truth",
    constraints="s3://sagemaker-us-east-1-381492271173/monitor/heart/baseline-results/constraints.json",
    schedule_cron_expression=CronExpressionGenerator.now(),
    data_analysis_start_time="-PT1H",
    data_analysis_end_time="-PT0H",
    enable_cloudwatch_metrics=True,
)



In [52]:
model_quality_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:381492271173:monitoring-schedule/monitoring-schedule-2024-03-21-01-19-31-499',
 'MonitoringScheduleName': 'monitoring-schedule-2024-03-21-01-19-31-499',
 'MonitoringScheduleStatus': 'Pending',
 'MonitoringType': 'ModelQuality',
 'CreationTime': datetime.datetime(2024, 3, 21, 1, 19, 32, 302000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 3, 21, 1, 19, 32, 395000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'NOW',
   'DataAnalysisStartTime': '-PT1H',
   'DataAnalysisEndTime': '-PT0H'},
  'MonitoringJobDefinitionName': 'model-quality-job-definition-2024-03-21-01-19-31-499',
  'MonitoringType': 'ModelQuality'},
 'EndpointName': 'sagemaker-scikit-learn-2024-03-21-00-03-05-484',
 'ResponseMetadata': {'RequestId': '1f602727-62e3-4d01-a872-ff411d4ab47b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1f602727-62e3-4d01-a872-ff411d4ab47b',
   'content-type': 

In [27]:
model_quality_monitor.list_executions()[-1].describe()

No executions found for schedule. monitoring_schedule_name: monitoring-schedule-2024-03-20-01-49-25-076


IndexError: list index out of range

In [53]:
#model_quality_monitor.delete_monitoring_schedule()


Deleting Monitoring Schedule with name: monitoring-schedule-2024-03-21-01-19-31-499
